In [1]:
import cv2
import numpy as np

In [2]:
image_path = "datasets/flower_photos/roses/159079265_d77a9ac920_n.jpg"

In [3]:
# Step 1: Load the image
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError("Image not found.")
original_image = image.copy()

In [4]:
mask = np.zeros(image.shape[:2], dtype="uint8")

In [4]:
def img_show(image=cv2.typing.MatLike, title= str):
    # Save or display the result
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
img_show(original_image, "Original Image")

In [7]:
# Step 2: Automatically detect object region (using simple thresholding here for demonstration)
gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

In [8]:
img_show(binary, "Binary Image")

In [8]:
# Find contours to get a bounding box
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)

In [9]:
x, y, w, h = cv2.boundingRect(largest_contour)

# Step 3: Initialize mask for GrabCut
mask = np.zeros(original_image.shape[:2], dtype=np.uint8)
rect = (x, y, w, h)

In [39]:
img_show(mask, "Binary Image")

In [10]:
# Step 4: Allocate background and foreground models
bg_model = np.zeros((1, 65), dtype=np.float64)
fg_model = np.zeros((1, 65), dtype=np.float64)

In [12]:
img_show(bg_model, "Binary Image")

In [11]:
import time

In [12]:
# apply GrabCut using the the bounding box segmentation method
start = time.time()
(mask, bg_model, fg_model) = cv2.grabCut(image, mask, rect, bg_model,
                                       fg_model, iterCount=10, mode=cv2.GC_INIT_WITH_RECT)
end = time.time()
print("[INFO] applying GrabCut took {:.2f} seconds".format(end - start))

[INFO] applying GrabCut took 1.14 seconds


In [13]:
# the output mask has for possible output values, marking each pixel
# in the mask as (1) definite background, (2) definite foreground,
# (3) probable background, and (4) probable foreground
values = (
	("Definite Background", cv2.GC_BGD),
	("Probable Background", cv2.GC_PR_BGD),
	("Definite Foreground", cv2.GC_FGD),
	("Probable Foreground", cv2.GC_PR_FGD),
)
# loop over the possible GrabCut mask values
for (name, value) in values:
	# construct a mask that for the current value
	print("[INFO] showing mask for '{}'".format(name))
	valueMask = (mask == value).astype("uint8") * 255
	# display the mask so we can visualize it
	cv2.imshow(name, valueMask)
	cv2.waitKey(0)

[INFO] showing mask for 'Definite Background'
[INFO] showing mask for 'Probable Background'
[INFO] showing mask for 'Definite Foreground'
[INFO] showing mask for 'Probable Foreground'


In [14]:
# we'll set all definite background and probable background pixels
# to 0 while definite foreground and probable foreground pixels are
# set to 1
outputMask = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD),
                      0, 1)
# scale the mask from the range [0, 1] to [0, 255]
outputMask = (outputMask * 255).astype("uint8")
# apply a bitwise AND to the image using our mask generated by
# GrabCut to generate our final output image
output = cv2.bitwise_and(image, image, mask=outputMask)

In [15]:
# show the input image followed by the mask and output generated by
# GrabCut and bitwise masking
cv2.imshow("Input", image)
cv2.imshow("GrabCut Mask", outputMask)
cv2.imshow("GrabCut Output", output)
cv2.waitKey(0)

27

In [16]:
img_show(c, "Binary Image")

In [17]:
# Convert the mask to binary format
mask_2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
segmented_image = original_image * mask_2[:, :, np.newaxis]

In [43]:
print(np.any(mask == 0))

True


In [ ]:
# Example usage
segmented_result = automatic_grabcut(image_path)

# Save or display the result
cv2.imwrite("segmented_output.jpg", segmented_result)
cv2.imshow("Segmented Image", segmented_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
import cv2
import numpy as np


def automatic_grabcut(image_path):
    """
    Perform automatic GrabCut segmentation on an image.

    Args:
    - image_path: Path to the input image.

    Returns:
    - segmented_image: The image with the background removed.
    """
    # Step 1: Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image at {image_path} not found.")
    original_image = image.copy()

    # Step 2: Convert to grayscale for better processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 3: Apply thresholding to roughly isolate the object
    _, binary = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

    # Step 4: Find contours to get a bounding box
    contours, _ = cv2.findContours(
        binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError(
            "No contours found. Ensure the image has distinguishable foreground and background.")

    # Get the largest contour (assuming the object occupies the largest area)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Step 5: Initialize the mask for GrabCut
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    rect = (x, y, w, h)

    # Mark probable foreground in the bounding box area
    mask[y:y + h, x:x + w] = cv2.GC_PR_FGD

    # Step 6: Allocate background and foreground models
    bg_model = np.zeros((1, 65), dtype=np.float64)
    fg_model = np.zeros((1, 65), dtype=np.float64)

    # Step 7: Apply GrabCut with the mask initialization
    cv2.grabCut(image, mask, rect, bg_model, fg_model,
                iterCount=5, mode=cv2.GC_INIT_WITH_MASK)

    # Step 8: Create the final binary mask
    mask_2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    # Step 9: Apply the mask to the original image
    segmented_image = original_image * mask_2[:, :, np.newaxis]

    return segmented_image


segmented_image = automatic_grabcut(image_path)

# Save and display the result
cv2.imwrite("segmented_output.jpg", segmented_image)
cv2.imshow("Segmented Image", segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
img_show(image, "Segmented Image")